In [1]:
from PIL import Image
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from pathlib import Path
import os
from tqdm import tqdm
import cv2
from torchvision import transforms as T
from pytorch_msssim import ssim, ms_ssim
import lpips

/home/soon/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
root = Path('../lightning_logs/inshop_2/')
sample_mask_root = root/'samples_mask'
sample_root = root/'samples'
gt_mask_root = root/'gt_mask'
gt_root = root/'gt'
src_root = root/'src'

masked_sample_root = root/'new_samples'
os.makedirs(str(masked_sample_root), exist_ok=True)

device='cuda:1'

loss_fn_vgg = lpips.LPIPS(net='vgg').to(device)

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/home/soon/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/soon/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/soon/miniconda3/envs/disco/lib/python3.8/site-packages/lpips/weights/v0.1/vgg.pth


In [6]:
sample_mask_files = glob(str(sample_mask_root/'*.png'),recursive=True)
fnames = [os.path.basename(f) for f in sample_mask_files]
len(fnames)

8107

In [7]:
ssim_scores = []
lpips_scores = []
msssim_scores =[]

for fname in tqdm(fnames[:]):
    sample_mask = cv2.imread(str(sample_mask_root/fname))
    sample = cv2.imread(str(sample_root/fname))
    maskedImg = cv2.bitwise_and(src1=sample, src2=sample_mask)
    
    gt = cv2.imread(str(gt_root/fname))
    #plt.imshow(gt[:,:,::-1])
    #plt.show()
    gt_mask = cv2.imread(str(gt_mask_root/fname), 0)    
    bg_pixel = gt[gt_mask==1].mean(0)
    gt[gt_mask!=0] = bg_pixel  
    bg = cv2.bitwise_and(src1=gt, src2=~sample_mask)
    new_sample = bg + maskedImg
    cv2.imwrite(str(masked_sample_root/fname),new_sample)
    #plt.imshow(new_sample[:,:,::-1])
    #plt.show()
    
    gt_tensor = T.ToTensor()(gt).unsqueeze(0).to(device)
    sample_tensor = T.ToTensor()(new_sample).unsqueeze(0).to(device)
    
    lpips_score = loss_fn_vgg(gt_tensor, sample_tensor).view(-1).detach().cpu().numpy().tolist()
    lpips_scores.extend(lpips_score)
    
    
    msssim_scores.extend(ms_ssim(gt_tensor, sample_tensor, data_range=1, size_average=False).cpu().numpy().tolist())
    ssim_scores.extend(ssim(sample_tensor, gt_tensor, data_range=1, size_average=False).cpu().numpy().tolist())
    
print("SSIM", np.mean(ssim_scores))    
print("MSSIM", np.mean(msssim_scores))
print("LPIPS", np.mean(lpips_scores))

100%|█████████████████████████████████████████████████████████████████████████████████████████| 8107/8107 [04:02<00:00, 33.43it/s]

SSIM 0.6649504051474501
MSSIM 0.40103516117679405
LPIPS 0.3312355726810276


In [8]:
import pickle
file_path = 'deepfashion_eval.p'
with open(file_path, 'wb') as file:
    pickle.dump({'ssim':ssim_scores, 'lpips':lpips_scores, 'mssim':msssim_scores}, file)